# Import useful stuff

In [1]:
# interactive plots in Jupyter, used to show plots inline in the notebook
%matplotlib inline

# The igraph library
from igraph import *

# Numpy for enhanced math array management
import numpy as np

#Usata per calcolare il logaritmo
from math import log

# statistical tools (we only use ECDF)
from statsmodels.distributions.empirical_distribution import ECDF

# Mathematical plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# use to control whether to show the entire cell output or only the last_expr (default)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to generate random numbers
from random import *

# to fit power law distributions
from powerlaw import *

# INTRODUZIONE
### Facebook (WOSN) is undirected network contains friendship data of Facebook users. 
### A node represents a user and an edge represents a friendship between two users.

# Parameters

In [22]:
#Select the dataset
#dataset="./facebook.ncol"
#dataset="./node.ncol"
#dataset="./dataset/socfb-wosn-friends.cut.edges" #BIGGEST        
#dataset="./dataset/socfb-wosn-friends.cut.sb.43K.edges" #MEDIUM
dataset="./dataset/socfb-wosn-friends.cut.sb.10K.edges" #MINOR

#Select if the dataset is directed or undirected
#direct=True
direct=False

#Select if connection-mode is weak or or strong
#This is the way to find Giant component
conn_mode="WEAK"
#conn_mode="STRONG"

# Punto 1 e Punto 2
## Reading graphs from files & data frames / Writing graphs to files

In [23]:
file_graph = read(dataset, format = "ncol", directed = direct)
summary(file_graph, verbosity=1, max_rows = 25, edge_list_format = 'edgelist')

IGRAPH UN-- 10000 229693 -- 
+ attr: name (v)
+ edges (vertex names):
        edge  
[0]    5--22  
[1]    5--683 
[2]    5--821 
[3]    5--469 
[4]    5--639 
[5]    5--814 
[6]    5--1528
[7]    5--1640
[8]    5--1642
[9]    5--1643
[10]   5--1648
[11]   5--2548
[12]   5--4246
[13]   5--4503
[14]   5--4964
[15]   9--23  
[16]   9--2058
[17]   9--309 
[18]   9--2175
[19]   469--9 
[20]   639--9 
[21]   1528--9
[22]   1640--9
[23]   9--47  
[24]   9--100 


# Punto 3.a)
### Il Giant Component esiste?

In [24]:
# Check whether the graph is connected or not
if file_graph.is_connected(mode = "STRONG")==True:
    print ("The graph is STRONGLY CONNECTED. \n\nNodes are mutally connected. All Graph is the Giant Component")
    giant_component=file_graph
elif file_graph.is_connected(mode = "WEAK")==True:
    print ("The graph is WEAKLY CONNECTED. \n\nNodes are connected. All Graph is the Giant Component")
    giant_component=file_graph
else:
    print ("The graph is NOT CONNECTED. \n\nIt is necessary to find the Giant Component")    
    # Compute the connected components in the graph
    #   - "WEAK" does not consider the direction of edges
    #All clusters
    clusters_list = file_graph.clusters(mode = conn_mode)
    # the number of clusters
    print ("Number of clusters: ",len(clusters_list))    
    # the membership of vertices in the clusters. Every pos has the cluster_ID associated to the vertex
    #clusters_list.membership[0:10]
    # the sizes of the clusters
    #print ("Clusters size:")
    #clusters_list.sizes()[0:10]

    #Does Giant COmponent exist? 
    #GC esiste se contiene una frazione >> di nodi rispetto a logN (N =numero totale di nodi nel grafo), 
    #Gli altri componenti sono nell’ordine di logN

    
    #Trashold
    trashold=10*log(file_graph.vcount(),10)
    print ("Trashold is:", trashold)
    
    
    # sizes (sorted, first 20 elements)
    sorted_clusters = sorted(clusters_list.sizes(), reverse=True)
    print("Lista dei cluster ordinata per dimensione:")
    sorted_clusters[0:19]
    
    
    #bigger_cluster=max(clusters_list.sizes())
    print ("Bigger Cluster:", sorted_clusters[0])
    print ("2nd Bigger Cluster:", sorted_clusters[1])
    
    if sorted_clusters[0] > trashold and sorted_clusters[1] < trashold:
        print ("GIANT COMPONENT EXIST!!!")
    
        #Select the Giant Componet (the biggest cluster)
        giant_component = clusters_list.giant()
    
        # number of vertices and edges in the original graph
        print ("Numero di nodi del giant componet ", giant_component.vcount())
        print ("Numero di nodi totali ", file_graph.vcount())
        print ("GC: ",giant_component.vcount()*100/file_graph.vcount(),"%")
    
        #From this time we consider GC as the only one cluster 
        #file_graph=giant_component
    else :
        print ("GIANT COMPONENT DOES NOT EXIST!!!")
        exit(0)



The graph is STRONGLY CONNECTED. 

Nodes are mutally connected. All Graph is the Giant Component


### Graph of major nodes (degree > 100)

In [25]:
# Select only nodes with degree > 100 from the Giant Component
"""
vs = fb_GC.vs.select(_degree_gt = 100)
fb_g_sub = fb_GC.induced_subgraph(vs)

try:
    del visual_style
    visual_style = {}
except NameError:
    visual_style = {}
    
visual_style["bbox"] = (600,600)
visual_style["label"] = []
visual_style["layout"] = fb_g_sub.layout_fruchterman_reingold()
visual_style["vertex_size"] = 5
visual_style["vertex_color"] = 'red'
visual_style["vertex_shape"] = 'circle'
visual_style["edge_arrow_size"] = 0.2
visual_style["edge_width"] = np.array(fb_g_sub.es["weight"])/10

plot(fb_g_sub, **visual_style)
"""

'\nvs = fb_GC.vs.select(_degree_gt = 100)\nfb_g_sub = fb_GC.induced_subgraph(vs)\n\ntry:\n    del visual_style\n    visual_style = {}\nexcept NameError:\n    visual_style = {}\n    \nvisual_style["bbox"] = (600,600)\nvisual_style["label"] = []\nvisual_style["layout"] = fb_g_sub.layout_fruchterman_reingold()\nvisual_style["vertex_size"] = 5\nvisual_style["vertex_color"] = \'red\'\nvisual_style["vertex_shape"] = \'circle\'\nvisual_style["edge_arrow_size"] = 0.2\nvisual_style["edge_width"] = np.array(fb_g_sub.es["weight"])/10\n\nplot(fb_g_sub, **visual_style)\n'

# Punto 3.b) Distribution Degree

In [ ]:
# degree() method
# - mode = "ALL" to consider the undirected graph
degree_sequnce = giant_component.degree(mode = "all")
#degree_sequnce[0:19]

# the maximum degree, and the ID of the node with maximum degree
print ("Degree MASSIMO:", max(degree_sequnce))
id_max = np.argmax(degree_sequnce)
print ("ID of the node with max degree: ",id_max)
print ("Degree MINIMO:", min(degree_sequnce))
print ("Grado medio: ",mean(degree_sequnce))




"""
# the set of neighbours of the node with max degree
# - NB: in case of bidirectional links, the same neighbour is counted twice if mode = 'all'
neighbours_of_major_node = giant_component.neighbors(id_max, mode="all")
neighbours_of_major_node[0:19]
len(neighbour_list_of_major_node)

# the set of nodes reachable from id_max with AT MOST 1 jump
neighbours_order_1 = giant_component.neighborhood(id_max, order = 1, mode="all")
neighbours_order_1[0:19]

# the number of such nodes
# - NB: it also includes the node id_max itself (which is reachable with 0 jumps)
# - thus, the number of nodes reachable with one jump is this - 1
len(neighbours_order_1)
giant_component.neighborhood_size(id_max, order = 1, mode="all")
"""

__Note__

Why is the output of $\tt{neighbourhood\_size()}$ different from the length of $\tt{nei}$?

- consider that we used a __directed__ graph, and think what does it means in terms of degree and neighbours

Let's redo the same on the equivalent __undirected__ graph

In [ ]:
"""# take the undirected version of the Giant Component 
# combine_edges tells what to do with the weights (default, lost attribute; here: sum values)
giant_component_undir = giant_component.as_undirected(combine_edges = "sum")

# Note the lower number of edges with respect to the directed version.
# This is because igraph automatically simplifies the graph (i.e., merges edges between the same nodes)
# to do so manually on a multi-edge graph: g.simplify()
# to check if the graph is simple or not: g.is_simple()
summary(giant_component_undir, verbosity = 1, edge_list_format = "edgelist", max_rows = 25)

# the maximum degree, and the ID of the node with maximum degree
degree_sequence_undir = giant_component_undir.degree()
max(degree_sequence_undir)
id_max_u = np.argmax(degree_sequence_undir)
id_max_u

# the set of neighbours of the node with max degree
neighbours_of_major_node_undir = giant_component_undir.neighbors(id_max_u)
len(neighbours_of_major_node_undir)

# the set of nodes reachable from id_max with AT MOST 1 jump
neighbours_order_1 = giant_component_undir.neighborhood(id_max, order = 1, mode="all")
neighbours_order_1[0:19]

# the number of such nodes
# - NB: it also includes the node id_max itself (which is reachable with 0 jumps)
# - thus, the number of nodes reachable with one jump is this - 1
len(neighbours_order_1)
giant_component_undir.neighborhood_size(id_max, order = 1, mode="all")
"""

### Degree density and CCDF

In [ ]:
dd_h, dd_h_bins, _ = plt.hist(degree_sequnce, bins=range(1,max(degree_sequnce)+2), density=True, color = 'red')

In [ ]:
# Degree density on a loglog scale
plt.loglog(dd_h_bins[:-1], dd_h, 'bo')
plt.xlabel("d")
plt.ylabel("P(Degree = d)")
plt.title("Degree density on a log-log scale")

In [ ]:
# Compute the CCDF - we can use 2 ways
# 1. use the histogram functions with parameter cumulative=-1 gives the CCDF
dd_h, dd_h_bins, _ = plt.hist(degree_sequnce, bins=range(1,max(degree_sequnce)+2), density = True, color = 'red', cumulative = -1)
plt.axis([0,400,0,1])
plt.xlabel("d")
plt.ylabel("CCDF(Degree > d)")
plt.title("CCDF primo metodo")

In [ ]:
# 2. More general: use the ECDF function of statsmodels.distributions.empirical_distribution
# ECDF(dataset) returns a the empirical CDF computed from the dataset, which can be used as a FUNCTION
# - i.e., it is possible to call ECDF(x) for any x, irrespective of the set of data from which the ECDF is derived
deg_cdf = ECDF(degree_sequnce)

# scale the fig size twice in length
default_sizes = plt.rcParams["figure.figsize"]
fig_sizes = (2*default_sizes[0], default_sizes[1])

# generate a figure with 2 subplots, organised in 1 row and 2 columns
# ax1 and ax2 ("axes") are used to access the individual plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = fig_sizes)

# plot the CCDF in lin-lin and log-log scales
# see http://matplotlib.org/api/axes_api.html for the API of the Axis class
# see http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot or the Axes.plot() documentation
# for the parameters of the plot method
degs = np.arange(1,max(degree_sequnce)+1)

ax1.plot(degs, 1-deg_cdf(degs), 'bo')
ax1.set_xlabel("$d$")
ax1.set_ylabel("$P(D>d)$")
ax1.set_title("Degree CCDF in a lin-lin scale")

ax2.loglog(degs, 1-deg_cdf(degs), 'bo')
ax2.set_xlabel("$d$")
ax2.set_ylabel("$P(D>d)$")
ax2.set_title("Degree CCDF in a log-log scale")

### Random graphs - Erdős–Rényi model

In [ ]:
# Now we compare the degree distributions for the complete Giant Component
er_p_GC = mean(giant_component_undir.degree())/giant_component_undir.vcount()
er_fb_all = Graph.Erdos_Renyi(giant_component_undir.vcount(), er_p_GC)

# take only the Giant Component
er_fb = er_fb_all.clusters(mode = "WEAK").giant()
print ("Il GC del modello di Erdos Renyl ha un numero di nodi ",er_fb.vcount()," minore o uguale del nostro grafo", giant_component_undir.vcount())

# we use GridSpecs for a finer control of the plot positioning
fig_sizes = (fig_sizes[0], 2*default_sizes[1])
f = plt.figure(figsize = fig_sizes)

# create a 2x2 Grid Specification
gs = gridspec.GridSpec(2, 2)

# add subplots to the figure, using the GridSpec gs
# position [0,0] (upper-left corner)
ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[0,1])
# the third plot spans the entire second row
ax3 = plt.subplot(gs[1,:])

# compute and plot the histogram of FB degrees
d_fb = giant_component_undir.degree()
_,_,_ = ax1.hist(d_fb, bins=range(1,max(d_fb)+2), density = True, color = 'red')
_ = ax1.set_xlim(0,80)
_ = ax1.set_xlabel("$d$")
_ = ax1.set_ylabel("Frequencies")
_ = ax1.set_title("Histogram of FB degrees")

# compute and plot the histogram of ER degrees
d_er = er_fb.degree()
_,_,_ = ax2.hist(d_er, bins=range(1,max(d_er)+2), density = True, color = 'blue')
_ = ax2.set_xlim(0,80)
_ = ax2.set_xlabel("$d$")
_ = ax2.set_ylabel("Frequencies")
_ = ax2.set_title("Histogram of ER degrees")

# compute and plot the degree CCDFs
fb_ecdf = ECDF(d_fb)
er_ecdf = ECDF(d_er)
x = np.arange(1,max(d_fb)+1)
_ = ax3.loglog(x, 1-fb_ecdf(x), 'ro', label = 'Facebook')
x = np.arange(1,max(d_er)+1)
_ = ax3.loglog(x, 1-er_ecdf(x), 'bo', label = 'Erdos-Renyi')
_ = ax3.set_xlabel("$d$")
_ = ax3.set_ylabel("$P(D>d)$")
_ = ax3.set_title("Comparison between degree CCDFs")
_ = ax3.legend(numpoints = 1)